# Importing Packages

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from collections import Counter
import bs4 as bs
import unicodedata
mdict = {"January":"1", "February":"2", "March":"3", "April":"4", "May":"5", "June":"6", "July":"7",\
         "August":"8", "September":"9", "October":"10", "November":"11", "December":"12"}
import us
STATES_DICT = us.states.mapping('name', 'abbr')

In [ ]:
governor = pd.read_csv(DATA_DIR/'cleaned'/'Clean_Governor_Winners.csv')
house = pd.read_csv(DATA_DIR/'cleaned'/'Clean_House_Winners.csv')
senate = pd.read_csv(DATA_DIR/'cleaned'/'Clean_Senate_Winners.csv')
nick = pd.read_pickle(DATA_DIR/'cleaned'/'tmp_for_nick.pkl')